In [1]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
url = "http://www.metacritic.com/game/pc/dishonored-2/user-reviews"
headers={'User-Agent':user_agent,} 

request = urllib.request.Request(url,None,headers) #The assembled request
data = urllib.request.urlopen(request).read()

In [4]:
soup = BeautifulSoup(data,"lxml")
#print(soup.prettify()[50000:100000])

Stuff of interest: <br>
div class="review_body" - the full review is here <br>
span class="blurb blurb_expanded" - if the review is long, the full review is here <br>
div class="review_grade" - 1-5 <br>
div class="date" - date of rating <br>
div class="name" - rater's name <br>

In [5]:
# get all the data associated with each review, where each element is a diff review
reviews = soup.findAll('li', attrs={'class': 'review user_review'})

In [5]:
# an example of a review where the text is in "span class=blurb blurb_expanded"
reviews[0]

<li class="review user_review" id="user_review_6803714">
<div class="review_btm review_btm_r"><div class="review_btm review_btm_l"><div class="review_top review_top_r"><div class="review_top review_top_l">
<div class="review_content">
<div class="review_section">
<div class="review_stats">
<div class="review_critic">
<div class="name">
<a href="/user/GoldieFoxx">GoldieFoxx</a>
</div>
<div class="date">Nov 10, 2016</div>
</div>
<div class="review_grade">
<div class="metascore_w user medium game mixed indiv">5</div>
</div>
</div>
<div class="review_body">
<span class="inline_expand_collapse inline_collapsed" id="review_blurb_6803714"><span class="blurb blurb_collapsed">This game gives me very mixed feelings. The story, graphics and gameplay are all fantastic and a great step up from the original.
<br/> <br/>It has a fantastic feel to it and if you enjoyed the original you will certainly enjoy this one.
If that was it, I would have easily given this a 9/10

BUT

There are so many things d

**Note:** Not all of the review divs have a blurb, so need to loop through each review and individually retrieve data for that row of the upcoming DB. This will make sure everything aligns nicely (e.g., user corresponds to who actually wrote the review")

In [6]:
# TODO: edit the review loop to also reflect the game title

review_text = []
for pos, review in enumerate(reviews):
    
    # Reviewer's username
    name = review.find('div', attrs={'class':'name'}).get_text().strip()
    
    # Date of Review
    date_text = review.find('div', attrs={'class':'date'}).get_text().strip()
    date_obj = datetime.strptime(date_text, '%b %d, %Y').date()
    
    # Review Grade
    grade = int(review.find('div', attrs={'class':'review_grade'}).get_text().strip())
    
    # Text body of the review, handling the 2 types of cases you can encounter
    rt = review.find('span', attrs={'class': 'blurb blurb_expanded'})
    if rt == None:
        rt = review.find('span')
    rt = rt.get_text()
    review_text.append((pos, name, date_obj, grade, rt))

In [7]:
reviews[4]

<li class="review user_review" id="user_review_6804977">
<div class="review_btm review_btm_r"><div class="review_btm review_btm_l"><div class="review_top review_top_r"><div class="review_top review_top_l">
<div class="review_content">
<div class="review_section">
<div class="review_stats">
<div class="review_critic">
<div class="name">
<a href="/user/eldumloido">eldumloido</a>
</div>
<div class="date">Nov 11, 2016</div>
</div>
<div class="review_grade">
<div class="metascore_w user medium game negative indiv">3</div>
</div>
</div>
<div class="review_body">
<span>Bad optimization. Getting headache from all the stuttering so can't really play the game. i7 6700k, gtx 1080, 32gb ram, samsung 950 pro ssd and running at 1080p at 60fps is not possible. This is why they wont give review copies anymore.</span>
</div>
</div>
<div class="review_section review_actions ">
<ul class="review_actions">
<li class="review_action review_helpful">
<div class="review_helpful">
<div class="rating_thumbs">
<

In [34]:
review_text

[(0,
  'GoldieFoxx',
  datetime.date(2016, 11, 10),
  5,
  "This game gives me very mixed feelings. The story, graphics and gameplay are all fantastic and a great step up from the original.\r It has a fantastic feel to it and if you enjoyed the original you will certainly enjoy this one.\rIf that was it, I would have easily given this a 9/10\r BUT\r There are so many things dragging it down right now. On PC anyway\r -Optimisation is terrible, with an i7 and GTX 1080, at 1080p im struggling to even maintain a 50-60fps framerate on ultra settings, with anti-aliasing at TXAA 1x\r -Mouse sensitivity is tied to FPS, so it is extremely hard to aim, especially with how jumpy and unpredictable the FPS is currently.\r -Pre-order bonuses. For most this doesnt matter, but trying desperately to tempt people into pre-ordering is always a bad sign in my opinion. And in this case it seems like they were doing it because they knew reviews would be mixed with its current state.\r Overall, a very good a

In [32]:
datetime.strptime(review_text[0][2], '%b %d, %Y').date()

datetime.date(2016, 11, 10)

### Example code from Analytics Vidhya

In [12]:
doc1 = "Sugar is bad to consume. My sister likes to have sugar, but not my father."
doc2 = "My father spends a lot of time driving my sister around to dance practice."
doc3 = "Doctors suggest that driving may cause increased stress and blood pressure."
doc4 = "Sometimes I feel pressure to perform well at school, but my father never seems to drive my sister to do better."
doc5 = "Health experts say that Sugar is not good for your lifestyle."

# compile documents
doc_complete = [doc1, doc2, doc3, doc4, doc5]

In [13]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in doc_complete] 

In [37]:
doc_clean

[['sugar', 'bad', 'consume', 'sister', 'like', 'sugar', 'father'],
 ['father',
  'spends',
  'lot',
  'time',
  'driving',
  'sister',
  'around',
  'dance',
  'practice'],
 ['doctor',
  'suggest',
  'driving',
  'may',
  'cause',
  'increased',
  'stress',
  'blood',
  'pressure'],
 ['sometimes',
  'feel',
  'pressure',
  'perform',
  'well',
  'school',
  'father',
  'never',
  'seems',
  'drive',
  'sister',
  'better'],
 ['health', 'expert', 'say', 'sugar', 'good', 'lifestyle']]

## MVP Cleaning of a few reviews

In [25]:
reviews_subset = [review_text[i][4] for i in range(len(review_text))]

In [26]:
reviews_clean = [clean(rev).split() for rev in reviews_subset] 

In [20]:
from gensim.models.ldamodel import LdaModel
from gensim import corpora

In [27]:
dictionary = corpora.Dictionary(reviews_clean)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in reviews_clean]

In [28]:
ldamod = LdaModel(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)

In [31]:
print(ldamod.print_topics(num_topics=3, num_words=10))

[(0, '0.029*"game" + 0.009*"2" + 0.009*"like" + 0.008*"dishonored" + 0.008*"time" + 0.006*"get" + 0.005*"level" + 0.005*"around" + 0.005*"1" + 0.005*"really"'), (1, '0.011*"game" + 0.009*"new" + 0.008*"dishonored" + 0.007*"way" + 0.007*"ending" + 0.006*"like" + 0.006*"charm" + 0.006*"one" + 0.005*"2" + 0.005*"it"'), (2, '0.040*"game" + 0.009*"even" + 0.009*"it" + 0.008*"story" + 0.008*"issue" + 0.008*"pc" + 0.008*"great" + 0.008*"fps" + 0.008*"really" + 0.007*"play"')]
